## Model Selection and Fitting

In [39]:
import pandas as pd
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,MultiComparison)
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [40]:
df = pd.read_csv('~/Desktop/merged_data.csv')

df=df.drop(columns=['Unnamed: 0'])

### Variables Description

- PlayerID: A numerical number range from 1 to 17, indicating a unique playerID for each player
- SessionType: A categorical variable describing the training session types: Skills, Mobility/Recovery, Game, Conditioning, Strength, Combat, Speed
- Duration: A float representing session length, ranging from 3.00 to 120.00
- RPE: An integer representing rate of perceived exertion (0-10 scale)
- SessionLoad: A float obtained by Duration * RPE
- DailyLoad: A float that is the sum of SessionLoad for a given day
- AcuteLoad: A float that is the average daily load over past 7 days
- ChronicLoad: A float that is the average daily load over past 30 days
- AcuteChronicRatio: A float that is calculated by AcuteLoad/ChronicLoad
- Load_Status: A categorical variable, which is dependent on AcuteChronicRatio.If ratio>1.2, status is high. If ratio<0.8, status is recovering. In between it is normal
- GameID: An integer representing a unique game ID, there are 38 games in total
- game_date: A date that indicates the date of the game
- train_date: A date that indicates the training date, which is one day ahead of game day
- AccelImpulse: A float that is the max absolute value of change in speed divided by change in time
- AccelLoad: A float that indicates the max load detected by the accelerometer
- Speed: A float that is the max movement speed of the player, in meters per second
- PerformanceScore: A measurement that is the weighted average of AccelImpulse, AccelLoad, and Speed. This measurement is considered as reponse variable in our model. It indicates how good the performance is for each player
- Outcome: A binary categorical variable, W indicates win, and L indicates loss.
- PointsDiff: An integer, which is the difference of scores. A positive difference indicates winning by how many points.
- Pain: A binary categorical variable(Yes or No) indicates whether a player is in pain
- llness: A categorical variable indicates whether a player is feeling ill, all possible values are  Yes, No, and Slightly Off
- Menstruation: A binary categorical variable(Yes or No) indicates whether a player is currently menstruating. 
- Nutrition: A binary categorical variable(Excellent or Okay)
- NutritionAdjustment: A binary categorical varibale indicates whether the player has made a nutrition adjustment that day
- EWMScore: A float number that measures the exponential moving average of wellness score

### Model Selection

   The typical regression model assumes all rows should be independent. However, in our data set, we have multiple observations(rows) for each player. These rows are dependent. We should not use typical linear regression.
   
   Linear mixed models are an extension of simple linear models to allow both fixed and random effects, and are particularly used when there is non independence in the data, such as arises from a hierarchical structure.
   
   When there are multiple levels, such as performances on different date from same player, the variability in the outcome can be thought of as being either within group or between group. Performance level observations are not independent, as within a given player performance are more similar.
   
   The advantages of using mixed:
   1. Keep as many rows of data as possible, instead of aggregating data by player. Although aggregate data analysis yields consistent and effect estimates and standard errors, it does not really take advantage of all the data, because player data are simply averaged.
   
   2. Even if we can run regression for each player, each model does not take advantage of the information in data from other players. This can cause some bias since we consider only for one specific player.
   
   3. It keeps the flexibility in treating time as continuous.
   
#### Linear Mixed Effects Model (LME)

A typical regression model: 

       Y = Xβ + ε, where X is fixed effects, ε is random noise

Linear Mixed Effects Model:

        Y = Xβ + Zu + ε 

Where y is a N×1 column vector of the outcome variables,
X is a N×p matrix of the p predictor variables,
β is a p×1 column vector of the fixed-effects regression coefficients,
Z is a N×q design matrix for random effects, 
u is a q×1 random complement to fixed β,
ε is a N×1 column vector of the residuals

#### Model Assumptions

1. Performance variance should be homogeneous for all players
2. Linearity(Residuals should be random)
3. Normality(Residuals should form a normal distribution

### Model Fitting

Our goad is to predict a player's overall performace based on variables from wellness, rpe, gps data as described above. We would like to build a multiple linear mixed effects model on variables that are significantly impacting player's performance. In order to achieve this, we applied Backward Elimination. We will firstly build a full model based on all variables we have. Consider the feature with the highest P-Value. If its P-value is greater than significance level (We fix 0.1 as significance level), we will eliminate this variable and build a new model. Until every variable has p-value < 0.1, our model is ready.

The reason for choosing Backward Elimination:
1. Although backward elimination and forward elimination will give the same result, by using backward elimination, we can manually reduce our model step by step, instead of running a built-in function and only getting the final model
2. By reducing on varibale at a time, we can also compare some other criteria to determine if the reduced model is as good as the previous model. For example, we can compare AIC, BIC, Likelihood, and Anova table.

In linear mixed effects model, in order to measure how good the model is, a useful technique is called REML – residual maximum likelihood. We will use this criteria to determine whether the reduced model is as good as full model.

Let's take a look at our full model

In [41]:
import statsmodels.api as sm

import statsmodels.formula.api as smf

full_md = smf.mixedlm("PerformanceScore  ~ SessionType +  Duration + RPE + SessionLoad + \
    DailyLoad + AcuteLoad + ChronicLoad + AcuteChronicRatio + Load_Status + \
        Outcome + PointsDiff + Pain + Illness + Menstruation + \
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = full_md.fit()
print(mdf.summary())

                    Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      PerformanceScore
No. Observations:      562          Method:                  REML            
No. Groups:            17           Scale:                   58.6021         
Min. group size:       3            Likelihood:              -1956.4791      
Max. group size:       80           Converged:               Yes             
Mean group size:       33.1                                                  
-----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------------
Intercept                         50.766    8.679  5.849 0.000  33.755 67.777
SessionType[T.Game]               -0.501    2.385 -0.210 0.834  -5.175  4.173
SessionType[T.Mobility/Recovery]   0.331    2.774  0.119 0.905  -5.106  5.769
Sessio

As we can see, ChronicLoad has p-value 0.992, which is significantly larger than 0.1, we will drop it for next model. Keep in mind, the REML is about 58.6.

In [42]:
md1 = smf.mixedlm("PerformanceScore  ~ SessionType +  Duration + RPE + SessionLoad + \
    DailyLoad + AcuteLoad +  AcuteChronicRatio + Load_Status + \
        Outcome + PointsDiff + Pain + Illness + Menstruation + \
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md1.fit()
print(mdf.summary())

                    Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      PerformanceScore
No. Observations:      562          Method:                  REML            
No. Groups:            17           Scale:                   58.4976         
Min. group size:       3            Likelihood:              -1953.0431      
Max. group size:       80           Converged:               Yes             
Mean group size:       33.1                                                  
-----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------------
Intercept                         50.784    8.618  5.893 0.000  33.893 67.674
SessionType[T.Game]               -0.500    2.380 -0.210 0.834  -5.165  4.165
SessionType[T.Mobility/Recovery]   0.334    2.765  0.121 0.904  -5.086  5.754
Sessio

As we can see, SessionType[T.Mobility/Recovery] has p-value 0.904, which is significantly larger than 0.1. Also, for the rest levels of this categorical variable, all p-value are greater than 0.1, we will drop it for next model. The REML is about 58.5.

In [43]:
md2 = smf.mixedlm("PerformanceScore  ~  Duration + RPE + SessionLoad + \
    DailyLoad + AcuteLoad +  AcuteChronicRatio + Load_Status + \
        Outcome + PointsDiff + Pain + Illness + Menstruation + \
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md2.fit()
print(mdf.summary())

                 Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    PerformanceScore
No. Observations:    562        Method:                REML            
No. Groups:          17         Scale:                 58.6317         
Min. group size:     3          Likelihood:            -1963.8942      
Max. group size:     80         Converged:             Yes             
Mean group size:     33.1                                              
-----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------
Intercept                   52.713    8.434  6.250 0.000  36.182 69.244
Load_Status[T.normal]       -6.701    2.449 -2.737 0.006 -11.500 -1.902
Load_Status[T.recovering]   -4.881    2.982 -1.637 0.102 -10.726  0.963
Outcome[T.W]                -2.659    1.315 -2.022 0.043  -5.236 -0.082
Pain[T.Ye

As we can see, Menstruation has p-value 0.903, which is significantly larger than 0.1, we will drop it for next model. The REML is about 58.6.

In [44]:
md3 = smf.mixedlm("PerformanceScore  ~  Duration + RPE + SessionLoad + \
    DailyLoad + AcuteLoad +  AcuteChronicRatio + Load_Status + \
        Outcome + PointsDiff + Pain + Illness +\
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md3.fit()
print(mdf.summary())

                 Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    PerformanceScore
No. Observations:    562        Method:                REML            
No. Groups:          17         Scale:                 58.5248         
Min. group size:     3          Likelihood:            -1965.0699      
Max. group size:     80         Converged:             Yes             
Mean group size:     33.1                                              
-----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------
Intercept                   52.513    8.271  6.349 0.000  36.301 68.724
Load_Status[T.normal]       -6.693    2.445 -2.737 0.006 -11.486 -1.900
Load_Status[T.recovering]   -4.864    2.976 -1.635 0.102 -10.696  0.968
Outcome[T.W]                -2.674    1.308 -2.045 0.041  -5.237 -0.111
Pain[T.Ye

Again, even if we have insigificant p-value for some levels in a categrical variable, we still need to keep it if there exist at least some significant levels. In this round, RPE will be gone. 
REML is about 58.5.


In [45]:
md4 = smf.mixedlm("PerformanceScore  ~  Duration +  SessionLoad + DailyLoad + AcuteLoad +  \
AcuteChronicRatio + Load_Status + Outcome + PointsDiff + Pain + Illness +\
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md4.fit()
print(mdf.summary())

                 Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    PerformanceScore
No. Observations:    562        Method:                REML            
No. Groups:          17         Scale:                 58.6464         
Min. group size:     3          Likelihood:            -1965.4590      
Max. group size:     80         Converged:             Yes             
Mean group size:     33.1                                              
-----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------
Intercept                   53.926    8.193  6.582 0.000  37.869 69.983
Load_Status[T.normal]       -6.552    2.445 -2.680 0.007 -11.344 -1.760
Load_Status[T.recovering]   -4.635    2.973 -1.559 0.119 -10.461  1.191
Outcome[T.W]                -2.662    1.309 -2.034 0.042  -5.228 -0.097
Pain[T.Ye

As we can see, SessionLoad has p-value 0.143, which is significantly larger than 0.05, we will drop it for next model. Keep in mind, the REML is about 58.6.

In [46]:
md5 = smf.mixedlm("PerformanceScore  ~  Duration +  DailyLoad + AcuteLoad +  \
AcuteChronicRatio + Load_Status + Outcome + PointsDiff + Pain + Illness +\
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md5.fit()
print(mdf.summary())

                Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   PerformanceScore
No. Observations:     562       Method:               REML            
No. Groups:           17        Scale:                58.7834         
Min. group size:      3         Likelihood:           -1961.9832      
Max. group size:      80        Converged:            Yes             
Mean group size:      33.1                                            
----------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------
Intercept                  55.121    8.159  6.756 0.000  39.131 71.112
Load_Status[T.normal]      -6.722    2.445 -2.750 0.006 -11.514 -1.931
Load_Status[T.recovering]  -4.914    2.970 -1.655 0.098 -10.735  0.907
Outcome[T.W]               -2.606    1.310 -1.989 0.047  -5.174 -0.038
Pain[T.Yes]            

As we can see, Duration has p-value 0.5393, which is significantly larger than 0.05, we will drop it for next model. Keep in mind, the REML is about 58.8.

In [47]:
md6 = smf.mixedlm("PerformanceScore  ~ DailyLoad + AcuteLoad +  \
AcuteChronicRatio + Load_Status + Outcome + PointsDiff + Pain + Illness +\
       Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md6.fit()
print(mdf.summary())

                 Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    PerformanceScore
No. Observations:    562        Method:                REML            
No. Groups:          17         Scale:                 58.7169         
Min. group size:     3          Likelihood:            -1958.8323      
Max. group size:     80         Converged:             Yes             
Mean group size:     33.1                                              
-----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-----------------------------------------------------------------------
Intercept                   54.331    8.054  6.746 0.000  38.546 70.116
Load_Status[T.normal]       -6.640    2.440 -2.722 0.006 -11.422 -1.859
Load_Status[T.recovering]   -4.826    2.965 -1.628 0.104 -10.636  0.984
Outcome[T.W]                -2.610    1.309 -1.994 0.046  -5.176 -0.044
Pain[T.Ye

In [48]:
md7 = smf.mixedlm("PerformanceScore  ~ DailyLoad + AcuteLoad +  Load_Status + Outcome + PointsDiff + \
Pain + Illness + Nutrition + NutritionAdjustment + EWMScore", df,groups = df["PlayerID"])
                 
mdf = md7.fit()
print(mdf.summary())

                Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   PerformanceScore
No. Observations:     562       Method:               REML            
No. Groups:           17        Scale:                58.9085         
Min. group size:      3         Likelihood:           -1962.3783      
Max. group size:      80        Converged:            Yes             
Mean group size:      33.1                                            
----------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------
Intercept                  58.509    7.704  7.594 0.000  43.409 73.609
Load_Status[T.normal]      -8.456    2.204 -3.836 0.000 -12.776 -4.136
Load_Status[T.recovering]  -8.037    2.309 -3.481 0.000 -12.563 -3.512
Outcome[T.W]               -2.655    1.311 -2.025 0.043  -5.225 -0.086
Pain[T.Yes]            

Up untill this model, our REML climb up to 58.9, and all variables or at least one level of categorical varibale have p-value <0.1, we confirm this is our final model.

Some Intepretations: 
1. The coefficients for two levels of Load_Status are about -8. The baseline level is high, so we can conclude if a player did a high load training before the game, their performance score is about 8 higher than normal traing or recovering.

2. The coefficients for two levels of Illness are about 5.6 and -9. The baseline level is No, so we can conclude if a player is ill, her performance score is lower by 9. It is noticable that if the player felt sightly off, her performance score is actually 5.67 higher.

3. EWMScore has coefficient -0.412, which indicates a negative relationship between performace and EWM score. This makes sense, since a high EWMScore means less soreness, fatigue. This indicates their training load is not high. Thus, it is consistent with first intepretation.